# IMPORTAR BIBLIOTECAS

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
#from google.colab import drive
from matplotlib.pyplot import figure
from sklearn.model_selection import train_test_split
from plotly import offline as py
from plotly import graph_objs as go
#from plotly import io as pio
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
#pio.renderers
#pio.renderers.default = "colab"

In [2]:
import plotly

# FUNÇÕES USO GERAL

In [3]:
def tic(name):
    global _start_time
    global _name
    _start_time = time.time()
    _name = name

def tac():
    t_sec = round(time.time() - _start_time)
    (t_min, t_sec) = divmod(t_sec,60)
    (t_hour,t_min) = divmod(t_min,60) 
    print('TIME '+_name+' : {}hour:{}min:{}sec'.format(t_hour,t_min,t_sec))
def fast_clean(df):
    booleandf = df.select_dtypes(include=[bool]).columns
    booleanDictionary = {True: 'true', False: 'false'}
    for column in booleandf:
      df[column] = df[column].map(booleanDictionary)
    df= df.astype(str)
    objectdf  = df.select_dtypes(include=[object]).columns
    for column in objectdf:
      df[column] = df[column].str.replace(' ', '')
      df[column] = np.where(df[column] == 'nan', str(df.index), df[column])
      df[column] = df[column].str.replace(',', '')
    indice = pd.Series(df.index, index=df['id']).drop_duplicates()
    df_id = pd.DataFrame({'id': df['id']})
    df = df.drop(['id'],axis=1)
    df2 = df_join(df) 
    df['id'] = df_id['id']
    df_clean = pd.DataFrame({'clean': df2})
    return(indice,df_clean)
def df_join(df):
    df2 = df.apply(lambda x: ','.join(x.astype(str)), axis=1)
    return(df2)


# FUNÇÕES DE TESTE

In [4]:
def metrica_test(df_pf):
    X = df_pf
    y = df_pf['id']
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size =0.2,random_state =50, shuffle = True )
    return(X_train, X_test, y_train, y_test)
def metrica_result(df_result,y_test):
    print("METRICA DE TESTE")
    result = []
    result_list2 = []
    result = df_result.loc[np.in1d(df_result['id'],y_test)]
    result_list2.append(result)
    result_final2 = pd.concat(result_list2)
    i1 = len(y_test)
    l1,l2 = df_result.shape
    f1,f2 = result_final2.shape
    print("TAMANHO DA BASE: "+str(i1))
    print("QUANTIDADE DE RECOMENDAÇÕES: "+str(l1))
    print("PERCENTUAL DE ITEM SOLICITADO NO RESULTADO: "+str(round(f1/i1*100,2))+'%')
    tac()

# FUNÇÃO PARA PLOTAR GRÁFICO DE ANÁLISE

In [5]:
def print_grafico(df_mkt,result_market,var):
    trace1 = go.Scatter(x = df_mkt[var].value_counts(normalize=True).sort_index().index,
                        y = df_mkt[var].value_counts(normalize=True).sort_index().values,
                        mode = 'markers+lines',
                        name = 'Porfolio',
                        line = {'dash': 'dot'})
    trace2 = go.Scatter(x = result_market[var].value_counts(normalize=True).sort_index().index,
                        y = result_market[var].value_counts(normalize=True).sort_index().values,
                        mode = 'markers+lines',
                        name = 'Resultado')
    data = [trace1, trace2]
    layout = go.Layout(
    title=go.layout.Title(
        text='Plot Title',
        xref='paper',
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text=var,
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='freq',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    )
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)

# CARREGAR ARQUVOS

In [6]:
tic('Carregar Arquivos')
#drive.mount('/gdrive')
#%cd /gdrive
cCaminho='./Data/'
df_mkt = pd.read_csv(cCaminho+'estaticos_market.csv',index_col = 0)
df_pf1 = pd.read_csv(cCaminho+'estaticos_portfolio1.csv',index_col= 0 )
df_pf2 = pd.read_csv(cCaminho+'estaticos_portfolio2.csv',index_col= 0 )
df_pf2 = pd.merge(df_pf2,df_mkt, on='id', how='inner')
df_pf3 = pd.read_csv(cCaminho+'estaticos_portfolio3.csv',index_col= 0 )
df_pf3 = pd.merge(df_pf3,df_mkt, on='id', how='inner')
tac()

TIME Carregar Arquivos : 0hour:0min:15sec


# TRATAMENTO DE NULOS

In [7]:
features = ['id', 'de_natureza_juridica', 'sg_uf', 'de_ramo', 'setor', 'idade_emp_cat', 
            'fl_st_especial', 'de_saude_tributaria', 'de_nivel_atividade']

In [8]:
def TratamentoDataFrame():
  portifolios = [df_mkt, df_pf1, df_pf2, df_pf3]
  for portifolio in portifolios:
    nome_port = portifolio
    #Aplicando a correção da base de dados para valores nulos
    def CorrigindoBase(coluna, df):
        if coluna != 'setor':
          df[coluna] = df[coluna].fillna('0')
        elif coluna == 'setor':
          df[coluna] = df[coluna].fillna('OUTROS')
    def ContagemValoresNulos(nome_df):
      matriz = [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
      count = 0
      for coluna in range(0, 9):
        nome_coluna = features[coluna]
        for valor in range(0, 3):
          if valor == 0:
            matriz[coluna][valor] = nome_coluna
          elif valor == 1:
            total_nulo = int(nome_df[nome_coluna].isnull().sum())
            matriz[coluna][valor] = total_nulo
          elif valor == 2:
            total_nulo = int(nome_df[nome_coluna].isnull().sum())
            total_df = int(nome_df['de_nivel_atividade'].shape[0])
            result = round(((total_nulo * 100)/(total_df))/100, 3)
            matriz[coluna][valor] = result
            if result > 0:
              CorrigindoBase(nome_coluna, nome_df)
    #Chamando a função de contagem de valores nulos      
    ContagemValoresNulos(nome_port)

In [9]:
TratamentoDataFrame()

# FUNÇÃO DE SIMILARIEDADE DE COSENO E KERNEL

In [10]:
def market_similarity(df, df_pf,columns,name,mode):
    print("START SIMILARITY " + name.upper()) 
    df = pd.DataFrame(df, columns=columns)
    df_pf = pd.DataFrame(df_pf, columns=columns)
    tic("CLEAN " + name.upper())
    indice, df_clean = fast_clean(df)
    indicepf, df_cleanpf = fast_clean(df_pf)
    tac()
    tic("TFID " + name.upper())
    tfidf = TfidfVectorizer(sublinear_tf=False,stop_words=None)
    tfidf_vector =  tfidf.fit(df_clean['clean'])
    tfidf_matrix = tfidf_vector.transform(df_clean['clean'])
    tfidf_matrixpf = tfidf_vector.transform(df_cleanpf['clean'])
    del df_clean
    del df_cleanpf
    tac()
    tic("RECOMMENDER " + name.upper())
    if mode=='cosine':
        similarity = cosine_similarity(tfidf_matrix,tfidf_matrixpf) 
    elif mode=='kernel':
        similarity = linear_kernel(tfidf_matrix,tfidf_matrixpf) 
    else:
        return    
    del tfidf_matrix
    del tfidf_matrixpf
    df_final = pd.DataFrame(data=similarity,columns=df_pf['id'])
    n,o= df_final.shape
    tic("ORDER " + name.upper())
    data_day_list = []
    for item_id in df_pf['id']:
        n_largest =df_final[item_id].nlargest(n + 1)
        n_largest = (n_largest[n_largest.values>0.8])
        for item in n_largest.index:
                idx = indice[item:item+1].index[0]
                if item_id!=idx:
                    data_day_list.append(idx)
    del df_final
    result = pd.DataFrame(list(data_day_list), columns=['id']).drop_duplicates()
    result_final = pd.merge(result,df, on='id', how='inner')
    result_final =  pd.DataFrame(result_final, columns=columns)
    result_final.to_csv(cCaminho+name+'.csv')
    tac()
    print("FINISH " + name.upper())
    return(result_final)

# FUNÇÃO DE SIMILARIEDADE DE KNN

In [11]:
def market_knn(df,df_pf,columns):
    def get_index_from_name(name):
        return df[df["id"]==name].index.tolist()[0]
    def print_similar_id(query=None):
        data_day_list = []
        found_id = get_index_from_name(query)
        for id in indices[found_id][1:]:
            data_day = (df.loc[id]["id"])
            data_day_list.append(data_day)
        return(data_day_list)
    print("START MARKET_KNN") 
    tic('CLEAN')  
    df_mktclean = pd.DataFrame(df, columns=columns)
    df_pf1clean = pd.DataFrame(df_pf, columns=columns)
    cons = pd.DataFrame({'colunas' : df_mktclean.columns,'tipo' : df_mktclean.dtypes})
    colunas_categoricas = list(cons[(cons['tipo'] == 'object') & (cons['colunas'] != 'id')]['colunas'])
    columns2 = []
    for coluna in colunas_categoricas:
        df_mktclean[coluna] = df_mktclean[coluna].astype('category')
        df_mktclean[coluna + '_cat'] = df_mktclean[coluna].cat.codes
        df_pf1clean[coluna] = df_pf1clean[coluna].astype('category')
        df_pf1clean[coluna + '_cat'] = df_pf1clean[coluna].cat.codes
        columns2.append(coluna + '_cat')
    df_mktclean = pd.DataFrame(df_mktclean, columns=columns2)
    df_pf1clean = pd.DataFrame(df_pf1clean, columns=columns2)
    X = df_mktclean.iloc[:, 1:9]
    y = df_mktclean.iloc[:, 0]
    Xp = df_pf1clean.iloc[:, 1:9]
    yp = df_pf1clean.iloc[:, 0]
    min_max_scaler = MinMaxScaler()
    market_features = min_max_scaler.fit_transform(X)  
    tac()
    tic("NEIGHBORS")
    nbrs = NearestNeighbors(n_neighbors=6 ,algorithm='kd_tree', n_jobs=-1).fit(market_features)
    tac()
    tic("DISTANCE")
    distances, indices = nbrs.kneighbors(market_features)
    tac()
    tic("RECOMMENDATION")
    result_list=[]
    data_day_list2 = []
    requis=df_pf['id']
    for element in requis:
        data_day_list = []
        data_day_list = (print_similar_id(query=element))
        for item in data_day_list:
            data_day = (df[df.id==item])
            data_day_list2.append(data_day)
        result = pd.concat(data_day_list2)
        result_list.append(result)
    result_final = pd.concat(result_list)
    result_final.drop_duplicates(keep = 'first', inplace = True)
    tac()
    print("FIM MARKET_KNN")
    return(result_final)

# VARIÁVEIS DEFINIDAS PARA UTILIZAÇÃO

In [12]:
columns = features

In [ ]:
print( 'INICIO COSINE')
result_metrica = market_similarity(df_mkt,df_pf1,columns,'PF1_cosine','cosine')
metrica_result(result_metrica,df_pf1['id'])
result_metrica = market_similarity(df_mkt,df_pf2,columns,'PF2_cosine','cosine')
metrica_result(result_metrica,df_pf2['id'])
result_metrica = market_similarity(df_mkt,df_pf3,columns,'PF3_cosine','cosine')
metrica_result(result_metrica,df_pf3['id'])
print('FIM COSINE')

INICIO COSINE
START SIMILARITY PF1_COSINE
TIME CLEAN PF1_COSINE : 0hour:0min:46sec
TIME TFID PF1_COSINE : 0hour:0min:9sec


In [ ]:
print( 'INICIO COSINE TESTE')
X_train, X_test, y_train, y_test = metrica_test(df_pf1)
result_metrica = market_similarity(df_mkt,X_train,columns,'Xtrain_cosine','cosine')
metrica_result(result_metrica,y_test)
X_train, X_test, y_train, y_test = metrica_test(df_pf2)
result_metrica = market_similarity(df_mkt,X_train,columns,'Xtrain_cosine','cosine')
metrica_result(result_metrica,y_test)
X_train, X_test, y_train, y_test = metrica_test(df_pf3)
result_metrica = market_similarity(df_mkt,X_train,columns,'Xtrain_cosine','cosine')
metrica_result(result_metrica,y_test)
print('FIM COSINE TESTE')

In [ ]:
print( 'INICIO KERNEL')
X_train, X_test, y_train, y_test = metrica_test(df_pf1)
result_metrica = market_similarity(df_mkt,X_train,columns,'Xtrain_cosine','kernel')
metrica_result(result_metrica,y_test)
X_train, X_test, y_train, y_test = metrica_test(df_pf2)
result_metrica = market_similarity(df_mkt,X_train,columns,'Xtrain_cosine','kernel')
metrica_result(result_metrica,y_test)
X_train, X_test, y_train, y_test = metrica_test(df_pf3)
result_metrica = market_similarity(df_mkt,X_train,columns,'Xtrain_cosine','kernel')
metrica_result(result_metrica,y_test)
print('FIM KERNEL')

print( 'INICIO KNN')
X_train, X_test, y_train, y_test = metrica_test(df_pf1)
result_metrica = market_knn(df_mkt,X_train,columns)
metrica_result(result_metrica,y_test)
X_train, X_test, y_train, y_test = metrica_test(df_pf2)
result_metrica = market_knn(df_mkt,X_train,columns)
metrica_result(result_metrica,y_test)
X_train, X_test, y_train, y_test = metrica_test(df_pf3)
result_metrica = market_knn(df_mkt,X_train,columns)
metrica_result(result_metrica,y_test)
print('FIM KNN')



# ANALISE GRÁFICA METÓDO COSINE - PORTIFÓLIO 1

In [ ]:
result_market = market_similarity(df_mkt,df_pf2,columns,'df3_cosine','cosine')
print_grafico(df_pf1,result_market,columns[1])
print_grafico(df_pf1,result_market,columns[2])
print_grafico(df_pf1,result_market,columns[3])
print_grafico(df_pf1,result_market,columns[4])
print_grafico(df_pf1,result_market,columns[5])
print_grafico(df_pf1,result_market,columns[6])
print_grafico(df_pf1,result_market,columns[7])
print_grafico(df_pf1,result_market,columns[8])
print_grafico(df_pf1,result_market,columns[9])

# ANALISE GRÁFICA METÓDO COSINE - PORTIFÓLIO 2

In [ ]:
result_market = market_similarity(df_mkt,df_pf2,columns,'df2_cosine','cosine')
print_grafico(df_pf2,result_market,columns[1])
print_grafico(df_pf2,result_market,columns[2])
print_grafico(df_pf2,result_market,columns[3])
print_grafico(df_pf2,result_market,columns[4])
print_grafico(df_pf2,result_market,columns[5])
print_grafico(df_pf2,result_market,columns[6])
print_grafico(df_pf2,result_market,columns[7])
print_grafico(df_pf2,result_market,columns[8])
print_grafico(df_pf2,result_market,columns[9])

# ANALISE GRÁFICA METÓDO COSINE - PORTIFÓLIO 3

In [ ]:
result_market = market_similarity(df_mkt,df_pf3,columns,'df3_cosine','cosine')
print_grafico(df_pf3,result_market,columns[1])
print_grafico(df_pf3,result_market,columns[2])
print_grafico(df_pf3,result_market,columns[3])
print_grafico(df_pf3,result_market,columns[4])
print_grafico(df_pf3,result_market,columns[5])
print_grafico(df_pf3,result_market,columns[6])
print_grafico(df_pf3,result_market,columns[7])
print_grafico(df_pf3,result_market,columns[8])
print_grafico(df_pf3,result_market,columns[9])